### Exercise

Use Melbourne_housing_FULL data to get an accuracy that's not causing an overfitting (don't forget to preprocess the data)

In [321]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import joblib

#### Exploaring the data

In [54]:
df = pd.read_csv("Melbourne_housing_FULL.csv")
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [55]:
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [57]:
df.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Price             2871
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom2            15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       740
YearBuilt          160
CouncilArea         33
Lattitude        13402
Longtitude       14524
Regionname           8
Propertycount      342
dtype: int64

In [58]:
df.shape

(34857, 21)

In [59]:
df.tail()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0
34856,Yarraville,3 Tarrengower St,2,h,1020000.0,PI,RW,24/02/2018,6.3,3013.0,...,1.0,0.0,250.0,103.0,1930.0,Maribyrnong City Council,-37.81810,144.89351,Western Metropolitan,6543.0


#### data preprocessing

In [60]:
df.columns[df.isna().any()]

Index(['Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [61]:
df.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
dtype: int64

drop bukding Area and Year Build too many null values

In [62]:
df2 = df.drop(['BuildingArea', 'YearBuilt', 'Date'], axis=1)
df2.shape

(34857, 18)

In [63]:
df2.Address.value_counts().sum()

34857

can't handle all this addresses

In [64]:
df3 = df2.drop(['Address'], axis=1)
df3.shape

(34857, 17)

In [65]:
df3.Postcode.unique()

array([3067., 3042., 3206., 3078., 3018., 3025., 3143., 3032., 3147.,
       3034., 3183., 3103., 3104., 3204., 3165., 3128., 3019., 3186.,
       3187., 3056., 3055., 3105., 3125., 3124., 3126., 3054., 3163.,
       3162., 3161., 3148., 3068., 3058., 3066., 3108., 3084., 3185.,
       3184., 3040., 3041., 3065., 3031., 3011., 3146., 3046., 3043.,
       3188., 3122., 3081., 3166., 3079., 3021., 3033., 3101., 3102.,
       3144., 3012., 3145., 3000., 3127., 3039., 3189., 3015., 3051.,
       3070., 3167., 3052., 3044., 3207., 3181., 3072., 3073., 3121.,
       3205., 3141., 3006., 3182., 3020., 3107., 3071., 3142., 3087.,
       3003., 3016., 3085., 3013., 3057., 3061., 3053., 3002., 3060.,
       3123., 3047., 3083., 3008., 3028., 3022., 3049., 3153., 3193.,
       3806., 3130., 3155., 3088., 3023., 3151., 3192., 3169., 3168.,
       3978., 3064., 3977., 3136., 3175., 3089., 3172., 3109., 3111.,
       3754., 3196., 3095., 3782., 3076., 3177., 3156., 3131., 3199.,
       3200., 3437.,

postcode is not an important feature to determine the price of the house

In [66]:
df4 = df3.drop(['Postcode'], axis=1)
df4.shape

(34857, 16)

In [67]:
df4.Car.unique()

array([ 1.,  0.,  2., nan,  6.,  5.,  4.,  3.,  8.,  7., 11., 26.,  9.,
       10., 18., 12.])

In [68]:
df4.Car.isnull().sum()

8728

also car is not an important feature

In [69]:
df5 = df4.drop('Car', axis=1)
df5.shape

(34857, 15)

Latitude and Longitude are the units that represent the coordinates at geographic coordinate system  so they are not important to us

In [70]:
df5.Longtitude

0        144.99580
1        144.99840
2        144.99340
3        145.01160
4        144.99440
           ...    
34852    144.88467
34853    144.88826
34854    144.87856
34855          NaN
34856    144.89351
Name: Longtitude, Length: 34857, dtype: float64

In [71]:
df6 = df5.drop(['Lattitude', 'Longtitude'], axis=1)
df6.shape

(34857, 13)

In [72]:
df6.isnull().sum()

Suburb               0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Distance             1
Bedroom2          8217
Bathroom          8226
Landsize         11810
CouncilArea          3
Regionname           3
Propertycount        3
dtype: int64

In [73]:
len(df6.Landsize.unique())

1685

In [74]:
len(df6.Propertycount.unique())

343

In [75]:
df6.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Bedroom2,Bathroom,Landsize,CouncilArea,Regionname,Propertycount
0,Abbotsford,2,h,NaN,SS,Jellis,2.5,2.0,1.0,126.0,Yarra City Council,Northern Metropolitan,4019.0
1,Abbotsford,2,h,1480000.0,S,Biggin,2.5,2.0,1.0,202.0,Yarra City Council,Northern Metropolitan,4019.0
2,Abbotsford,2,h,1035000.0,S,Biggin,2.5,2.0,1.0,156.0,Yarra City Council,Northern Metropolitan,4019.0
3,Abbotsford,3,u,NaN,VB,Rounds,2.5,3.0,2.0,0.0,Yarra City Council,Northern Metropolitan,4019.0
4,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3.0,2.0,134.0,Yarra City Council,Northern Metropolitan,4019.0


In [76]:
Bedroom2_mean = round(df.Bedroom2.mean())
Bathroom_mean = round(df.Bathroom.mean())
Landsize_mean = df.Landsize.mean()

df7 = df6.copy()
df7.Bedroom2.fillna(Bedroom2_mean, inplace = True)
df7.Bathroom.fillna(Bathroom_mean, inplace=True)
df7.Landsize.fillna(Landsize_mean, inplace = True)

df7.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Bedroom2,Bathroom,Landsize,CouncilArea,Regionname,Propertycount
0,Abbotsford,2,h,NaN,SS,Jellis,2.5,2.0,1.0,126.0,Yarra City Council,Northern Metropolitan,4019.0
1,Abbotsford,2,h,1480000.0,S,Biggin,2.5,2.0,1.0,202.0,Yarra City Council,Northern Metropolitan,4019.0
2,Abbotsford,2,h,1035000.0,S,Biggin,2.5,2.0,1.0,156.0,Yarra City Council,Northern Metropolitan,4019.0
3,Abbotsford,3,u,NaN,VB,Rounds,2.5,3.0,2.0,0.0,Yarra City Council,Northern Metropolitan,4019.0
4,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3.0,2.0,134.0,Yarra City Council,Northern Metropolitan,4019.0


In [77]:
df7.shape

(34857, 13)

In [78]:
df7.isnull().sum()

Suburb              0
Rooms               0
Type                0
Price            7610
Method              0
SellerG             0
Distance            1
Bedroom2            0
Bathroom            0
Landsize            0
CouncilArea         3
Regionname          3
Propertycount       3
dtype: int64

In [79]:
df7.dropna(inplace=True)
df7.shape

(27244, 13)

In [80]:
df7.isnull().sum()

Suburb           0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Distance         0
Bedroom2         0
Bathroom         0
Landsize         0
CouncilArea      0
Regionname       0
Propertycount    0
dtype: int64

In [81]:
df7.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Bedroom2,Bathroom,Landsize,CouncilArea,Regionname,Propertycount
1,Abbotsford,2,h,1480000.0,S,Biggin,2.5,2.0,1.0,202.0,Yarra City Council,Northern Metropolitan,4019.0
2,Abbotsford,2,h,1035000.0,S,Biggin,2.5,2.0,1.0,156.0,Yarra City Council,Northern Metropolitan,4019.0
4,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3.0,2.0,134.0,Yarra City Council,Northern Metropolitan,4019.0
5,Abbotsford,3,h,850000.0,PI,Biggin,2.5,3.0,2.0,94.0,Yarra City Council,Northern Metropolitan,4019.0
6,Abbotsford,4,h,1600000.0,VB,Nelson,2.5,3.0,1.0,120.0,Yarra City Council,Northern Metropolitan,4019.0


In [82]:
len(df7.Suburb.unique())

344

In [83]:
df7.Suburb.value_counts()

Reservoir         727
Bentleigh East    493
Richmond          439
Preston           415
Brunswick         387
                 ... 
Tecoma              1
croydon             1
Botanic Ridge       1
Wandin North        1
Eynesbury           1
Name: Suburb, Length: 344, dtype: int64

In [85]:
len(df7.CouncilArea.unique())

33

In [86]:
len(df7.Regionname.unique())

8

In [87]:
suburb_state = df7.groupby('Suburb')['Suburb'].agg('count').sort_values(ascending=False)

In [88]:
suburb_state

Suburb
Reservoir          727
Bentleigh East     493
Richmond           439
Preston            415
Brunswick          387
                  ... 
Eynesbury            1
Coldstream           1
Cranbourne East      1
Darley               1
viewbank             1
Name: Suburb, Length: 344, dtype: int64

In [90]:
len(suburb_state[suburb_state <= 10])

73

In [91]:
suburb_state_less_than_10 = suburb_state[suburb_state <= 10]

In [92]:
df8 = df7.copy()
df8.Suburb = df8.Suburb.apply(lambda x: 'Others' if x in suburb_state_less_than_10 else x)
len(df8.Suburb.unique())

272

In [93]:
df8.Type.unique()

array(['h', 't', 'u'], dtype=object)

In [95]:
df8.Type.replace(to_replace=['h', 't', 'u'], value=[1,2,3], inplace=True)

In [96]:
df8.Regionname.unique()

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria'], dtype=object)

In [98]:
df8.Regionname.replace(to_replace=['Northern Metropolitan', 'Western Metropolitan', 'Southern Metropolitan',
                                  'Eastern Metropolitan', 'South-Eastern Metropolitan', 'Eastern Victoria', 'Northern Victoria',
                                  'Western Victoria'], value=[1,2,3,4,5,6,7,8], inplace=True)

In [99]:
df8.Method.unique()

array(['S', 'SP', 'PI', 'VB', 'SA'], dtype=object)

In [100]:
df8.Method.replace(to_replace=['S', 'SP', 'PI', 'VB', 'SA'], value=[1,2,3,4,5], inplace=True)

In [102]:
len(df8.SellerG.unique())

349

In [103]:
df9 = df8.drop('SellerG', axis=1)
df9.shape

(27244, 12)

In [104]:
df9.head()

,Suburb,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Landsize,CouncilArea,Regionname,Propertycount
1,Abbotsford,2,1,1480000.0,1,2.5,2.0,1.0,202.0,Yarra City Council,1,4019.0
2,Abbotsford,2,1,1035000.0,1,2.5,2.0,1.0,156.0,Yarra City Council,1,4019.0
4,Abbotsford,3,1,1465000.0,2,2.5,3.0,2.0,134.0,Yarra City Council,1,4019.0
5,Abbotsford,3,1,850000.0,3,2.5,3.0,2.0,94.0,Yarra City Council,1,4019.0
6,Abbotsford,4,1,1600000.0,4,2.5,3.0,1.0,120.0,Yarra City Council,1,4019.0


In [105]:
len(df9.CouncilArea.unique())

33

In [109]:
dummies1 = pd.get_dummies(df9['Suburb'])
dummies1

,Abbotsford,Aberfeldie,Airport West,Albert Park,Albion,Alphington,Altona,Altona Meadows,Altona North,Ardeer,...,West Melbourne,Westmeadows,Wheelers Hill,Williamstown,Williamstown North,Windsor,Wollert,Wyndham Vale,Yallambie,Yarraville
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [110]:
dummies1.drop('Yarraville', axis=1, inplace=True)

In [111]:
dummies2 = pd.get_dummies(df9['CouncilArea'])
dummies2.head()

,Banyule City Council,Bayside City Council,Boroondara City Council,Brimbank City Council,Cardinia Shire Council,Casey City Council,Darebin City Council,Frankston City Council,Glen Eira City Council,Greater Dandenong City Council,...,Moorabool Shire Council,Moreland City Council,Nillumbik Shire Council,Port Phillip City Council,Stonnington City Council,Whitehorse City Council,Whittlesea City Council,Wyndham City Council,Yarra City Council,Yarra Ranges Shire Council
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [112]:
dummies2.drop('Yarra Ranges Shire Council', axis=1, inplace=True)
dummies2.head()

,Banyule City Council,Bayside City Council,Boroondara City Council,Brimbank City Council,Cardinia Shire Council,Casey City Council,Darebin City Council,Frankston City Council,Glen Eira City Council,Greater Dandenong City Council,...,Moonee Valley City Council,Moorabool Shire Council,Moreland City Council,Nillumbik Shire Council,Port Phillip City Council,Stonnington City Council,Whitehorse City Council,Whittlesea City Council,Wyndham City Council,Yarra City Council
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [113]:
df10 = pd.concat([df9, dummies1, dummies2], axis=1)
df10.head()

,Suburb,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Landsize,CouncilArea,...,Moonee Valley City Council,Moorabool Shire Council,Moreland City Council,Nillumbik Shire Council,Port Phillip City Council,Stonnington City Council,Whitehorse City Council,Whittlesea City Council,Wyndham City Council,Yarra City Council
1,Abbotsford,2,1,1480000.0,1,2.5,2.0,1.0,202.0,Yarra City Council,...,0,0,0,0,0,0,0,0,0,1
2,Abbotsford,2,1,1035000.0,1,2.5,2.0,1.0,156.0,Yarra City Council,...,0,0,0,0,0,0,0,0,0,1
4,Abbotsford,3,1,1465000.0,2,2.5,3.0,2.0,134.0,Yarra City Council,...,0,0,0,0,0,0,0,0,0,1
5,Abbotsford,3,1,850000.0,3,2.5,3.0,2.0,94.0,Yarra City Council,...,0,0,0,0,0,0,0,0,0,1
6,Abbotsford,4,1,1600000.0,4,2.5,3.0,1.0,120.0,Yarra City Council,...,0,0,0,0,0,0,0,0,0,1


In [114]:
df10.drop(['Suburb', 'CouncilArea'], axis=1, inplace=True)
df10.head()

,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Landsize,Regionname,Propertycount,...,Moonee Valley City Council,Moorabool Shire Council,Moreland City Council,Nillumbik Shire Council,Port Phillip City Council,Stonnington City Council,Whitehorse City Council,Whittlesea City Council,Wyndham City Council,Yarra City Council
1,2,1,1480000.0,1,2.5,2.0,1.0,202.0,1,4019.0,...,0,0,0,0,0,0,0,0,0,1
2,2,1,1035000.0,1,2.5,2.0,1.0,156.0,1,4019.0,...,0,0,0,0,0,0,0,0,0,1
4,3,1,1465000.0,2,2.5,3.0,2.0,134.0,1,4019.0,...,0,0,0,0,0,0,0,0,0,1
5,3,1,850000.0,3,2.5,3.0,2.0,94.0,1,4019.0,...,0,0,0,0,0,0,0,0,0,1
6,4,1,1600000.0,4,2.5,3.0,1.0,120.0,1,4019.0,...,0,0,0,0,0,0,0,0,0,1


I finished the preprocessing

#### Assign Features and target variables

In [115]:
X = df10.drop('Price', axis=1)
X.head()

,Rooms,Type,Method,Distance,Bedroom2,Bathroom,Landsize,Regionname,Propertycount,Abbotsford,...,Moonee Valley City Council,Moorabool Shire Council,Moreland City Council,Nillumbik Shire Council,Port Phillip City Council,Stonnington City Council,Whitehorse City Council,Whittlesea City Council,Wyndham City Council,Yarra City Council
1,2,1,1,2.5,2.0,1.0,202.0,1,4019.0,1,...,0,0,0,0,0,0,0,0,0,1
2,2,1,1,2.5,2.0,1.0,156.0,1,4019.0,1,...,0,0,0,0,0,0,0,0,0,1
4,3,1,2,2.5,3.0,2.0,134.0,1,4019.0,1,...,0,0,0,0,0,0,0,0,0,1
5,3,1,3,2.5,3.0,2.0,94.0,1,4019.0,1,...,0,0,0,0,0,0,0,0,0,1
6,4,1,4,2.5,3.0,1.0,120.0,1,4019.0,1,...,0,0,0,0,0,0,0,0,0,1


In [121]:
y = df10.Price

#### Spliting the data into trainning and testing data

In [317]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=95)

In [318]:
model = Lasso(alpha=50, max_iter=100, tol=0.1)
model.fit(X_train, y_train)

C:\Users\eyada\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1528636546949605.0, tolerance: 855390382222235.6
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=50, max_iter=100, tol=0.1)

In [319]:
model.score(X_train, y_train)

0.6498832898990272

In [320]:
model.score(X_test, y_test)

0.6729162214000205

#### Save the model

In [322]:
joblib.dump(model, 'Melbourne_housing prediciotn')

['Melbourne_housing prediciotn']

#### load the model

In [324]:
mj = joblib.load('Melbourne_housing prediciotn')

#### using the model after loading it

In [326]:
mj.predict(X_test)

array([ 462944.92782788, 3168492.77936967,  784228.53719491, ...,
        799727.17585087, 1610474.96447026, 1275408.91736238])